In [1]:
# conda install -c conda-forge gmaps

In [68]:
# import packages
import gmaps
import pandas as pd
from api_keys import g_key
import requests
import json
from pprint import pprint

In [69]:
# api_key
gmaps.configure(api_key=g_key)

In [70]:
# read data
data = pd.read_csv("retrieved_data.csv")

# store lat long in variable
coordinates = data[["lat","long"]]

# humidity
humidity = data["humidity"]
data.head()

,lat,long,closest city,country code,temp,humidity,cloudiness,wind speed
0,50.241814,-63.598569,havre-saint-pierre,ca,53.96,66,1,4.61
1,19.724111,-155.086819,hilo,us,71.42,85,20,5.75
2,-33.653158,115.345489,busselton,au,51.04,74,3,15.32
3,37.821037,-25.514814,ribeira grande,pt,73.80,53,20,10.36
4,36.148614,-95.990516,barrow,us,30.22,86,90,16.11


In [71]:
# map
figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius= 1)

# Add layer
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Ideal weather conditions

In [61]:
# filter for ideal weather conditions
data = data[data["temp"]  >= 68]
data = data[data["temp"]  <= 77]
data = data[data["wind speed"]  <= 77]
data =data[data["wind speed"]< 3]
data =data[data["cloudiness"]<50]

In [62]:
# geocoordinates
coordinates = "-31.9523, 115.8613"
search = "lodging"
radius = 5000

# set up a parameters dictionary
params = {
    "location": coordinates,
    "types": search,
    "radius": radius,
    "key": g_key
}

In [63]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()

In [64]:
data['Hotel Name'] = " "
search = "lodging"
radius = 5000
params = {
            "types": search,
            "radius": radius,
            "key": g_key
        }

# find hotel names
print("Starting search...")
for index, row in data.iterrows():
        
    coordinates = f"{row['lat']},{row['long']}"
    params["location"] = coordinates
    try:
        response = requests.get(base_url, params=params).json()
        data.loc[index,'Hotel Name']  =response['results'][0]['name']
    except (KeyError, IndexError):
        data = data.drop(index)
        print('Cant find hotel... skipping')
print('Search Done')

Starting search...
Search Done


In [73]:
# hotels
data

,lat,long,closest city,country code,temp,humidity,cloudiness,wind speed
0,50.241814,-63.598569,havre-saint-pierre,ca,53.96,66,1,4.61
1,19.724111,-155.086819,hilo,us,71.42,85,20,5.75
2,-33.653158,115.345489,busselton,au,51.04,74,3,15.32
3,37.821037,-25.514814,ribeira grande,pt,73.80,53,20,10.36
4,36.148614,-95.990516,barrow,us,30.22,86,90,16.11
5,-54.801912,-68.302951,ushuaia,ar,47.86,49,20,17.27
6,69.219812,-51.098603,ilulissat,gl,26.62,50,3,4.61
7,-5.093849,-81.096217,paita,pe,66.76,78,18,10.76
8,-12.582529,-69.193276,tambopata,pe,84.09,65,0,6.91
9,44.366788,-100.353752,pierre,us,73.85,62,1,1.99


In [72]:
# marker co ordinates
mark_cord = data[['lat', 'long']]


markers = gmaps.marker_layer(mark_cord)
figure.add_layer(markers)
figure

Figure(layout=FigureLayout(height='420px'))